In [137]:
import matplotlib
import pandas as pd
import requests
import io
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime
import re
from datetime import datetime


%matplotlib inline
    


%matplotlib inline
url1 = "https://raw.githubusercontent.com/docju/datasciencecapstone/main/results.csv"
download1 = requests.get(url1).content

df = pd.read_csv(io.StringIO(download1.decode('ISO-8859-1')))
df['date']=pd.to_datetime(df['DateTime'])
df['outcome']=df['FTHG']-df['FTAG']

def result(n):
    if n >0:
        res=1
    if n==0:
        res=0
    if n<0:
        res=-1
    return res

df['result']=df.apply(lambda x: result(x['outcome']),axis=1)
all_results=df[['Season','date','HomeTeam','AwayTeam','FTHG','FTAG','outcome','result']][~df['Season'].isin(['2020-21','2021-22'])]

print(all_results.head())

    Season       date     HomeTeam        AwayTeam  FTHG  FTAG  outcome  \
0  1993-94 1993-08-14      Arsenal        Coventry     0     3       -3   
1  1993-94 1993-08-14  Aston Villa             QPR     4     1        3   
2  1993-94 1993-08-14      Chelsea       Blackburn     1     2       -1   
3  1993-94 1993-08-14    Liverpool  Sheffield Weds     2     0        2   
4  1993-94 1993-08-14     Man City           Leeds     1     1        0   

   result  
0      -1  
1       1  
2      -1  
3       1  
4       0  


In [138]:
def split_season_train_test (season,test,input_df=all_results):
    '''
    This function takes data from a particular season and splits it into training and test samples
    
    INPUT:
    input_df - (dataframe) input matrix containing results of all seasons
    season - (string) season to split
    test - (int) number of rows that constitute the test data frame
    
    OUTPUT:
    season_train - (dataframe) a dataframe containing only the non-test rows
    season_test - (dataframe) a dataframe containing the test rows
    '''
    train=input_df[input_df['Season']==season].shape[0]-test
    season_train=input_df[input_df['Season']==season].copy().head(train)
    season_test=input_df[input_df['Season']==season].copy().tail(test)

    return season_train, season_test

train,test=split_season_train_test('2019-20',92)
print(test)

        Season                date          HomeTeam          AwayTeam  FTHG  \
10332  2019-20 2020-06-17 18:00:00       Aston Villa  Sheffield United     0   
10333  2019-20 2020-06-17 20:15:00          Man City           Arsenal     3   
10334  2019-20 2020-06-19 18:00:00           Norwich       Southampton     0   
10335  2019-20 2020-06-19 20:15:00         Tottenham        Man United     1   
10336  2019-20 2020-06-20 12:30:00           Watford         Leicester     1   
10337  2019-20 2020-06-20 15:00:00          Brighton           Arsenal     2   
10338  2019-20 2020-06-20 17:30:00          West Ham            Wolves     0   
10339  2019-20 2020-06-20 19:45:00       Bournemouth    Crystal Palace     0   
10340  2019-20 2020-06-21 14:00:00         Newcastle  Sheffield United     3   
10341  2019-20 2020-06-21 16:15:00       Aston Villa           Chelsea     1   
10342  2019-20 2020-06-21 19:00:00           Everton         Liverpool     0   
10343  2019-20 2020-06-22 20:00:00      

In [4]:
df2=covid_season_pre_break.copy()
output = np.asmatrix(df2.groupby(['HomeTeam', 'AwayTeam'])['outcome'].max().unstack())

df3=covid_season_post_break.copy()
output_post = np.asmatrix(df3.groupby(['HomeTeam', 'AwayTeam'])['outcome'].max().unstack())

 
 

NameError: name 'covid_season_pre_break' is not defined

In [140]:
def result_1(n):
    if n==1:
        res=3
    if n==0:
        res=1
    if n==-1:
        res=0
    return res


def table_create(season, scores=all_results):
    period=scores[all_results['Season']==season].copy()
    period['home_points']=period.apply(lambda x: result_1(x['result']),axis=1)
    period['away_points']=period.apply(lambda x: result_1(-1*x['result']),axis=1)
    period['home_gd']=period['outcome']
    period['away_gd']=-1*period['outcome']
    period['home_games']=1
    period['away_games']=1
    home_games=period.groupby('HomeTeam').sum()['home_games']
    away_games=period.groupby('AwayTeam').sum()['away_games']
    home_sum=period.groupby('HomeTeam').sum()['home_points']
    away_sum=period.groupby('AwayTeam').sum()['away_points']
    home_gd=period.groupby('HomeTeam').sum()['home_gd']
    away_gd=period.groupby('AwayTeam').sum()['away_gd']
    home_goals=period.groupby('HomeTeam').sum()['FTHG']
    away_goals=period.groupby('AwayTeam').sum()['FTAG']
    points_total=pd.concat([home_games,away_games,home_sum,away_sum, home_gd,away_gd,home_goals,away_goals],axis=1)
    points_total.index.name='Team'
    points_total=points_total.reset_index()
    points_total['Played']=points_total['home_games']+points_total['away_games']
    points_total['GF']=points_total['FTHG']+points_total['FTAG']
    points_total['GD']=points_total['home_gd']+points_total['away_gd']
    points_total['Points']=points_total['home_points']+points_total['away_points']
    points_total=points_total[['Played','Team','GF','GD','Points']].sort_values(['Points','GD','GF','Team'],ascending=[False,False,False,True])   
    
    return points_total

print(table_create('2019-20'))

train_matrix = np.asmatrix(train.groupby(['HomeTeam', 'AwayTeam'])['result'].max().unstack())

print(train_matrix)
    

    Played              Team   GF  GD  Points
9       38         Liverpool   85  52      99
10      38          Man City  102  67      81
11      38        Man United   66  30      66
5       38           Chelsea   69  15      66
8       38         Leicester   67  26      62
16      38         Tottenham   61  14      59
19      38            Wolves   51  11      59
0       38           Arsenal   56   8      56
14      38  Sheffield United   39   0      54
4       38           Burnley   43  -7      54
15      38       Southampton   51  -9      52
7       38           Everton   44 -12      49
12      38         Newcastle   38 -20      44
6       38    Crystal Palace   31 -19      43
3       38          Brighton   39 -15      41
18      38          West Ham   49 -13      39
1       38       Aston Villa   41 -26      35
2       38       Bournemouth   40 -25      34
17      38           Watford   36 -28      34
13      38           Norwich   26 -49      21
[[ nan   1.   1.  -1.   1.  -1.   

In [141]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=500):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) #-n_users
    
    # initialize the user and movie matrices with random values
    home_mat = np.random.rand(n_users, latent_features)
    away_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    #print(home_mat)
    #print(away_mat)
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
 
                
                # if the rating exists
                if pd.isna(ratings_mat[i,j]) is False:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(home_mat[i, :], away_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        home_mat[i, k] += learning_rate * (2*diff*away_mat[k, j])
                        away_mat[k, j] += learning_rate * (2*diff*home_mat[i, k])

        # print results for iteration
        if iteration in {0,1,iters-1}:
            print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return home_mat, away_mat 

In [142]:
home_mat,away_mat=FunkSVD(train_matrix,latent_features=20,learning_rate=0.005, iters=500)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 11.753115
2 		 2.684351
500 		 0.000005


In [160]:
result_matrix=np.dot(home_mat, away_mat)
home_results=result_matrix.copy()


for i in range(0,20):
    for j in range(0,20):
        if i==j: 
            home_results[i,j]=0
        elif i!=j and result_matrix[i,j]>=0:
            home_results[i,j]=1+2*result_matrix[i,j]
        elif i!=j and result_matrix[i,j]<0:
            home_results[i,j]=1+result_matrix[i,j]


def away_version(result_matrix):
    n=len(result_matrix)
    new_mat=result_matrix.copy()
    for i in range(n):
        for j in range(n):
            if i==j:
                new_mat[i,j]=0
            elif i!=j:
                new_mat[i,j]=-result_matrix[i,j]                
            
    new_mat_t=np.transpose(new_mat)
    return new_mat_t

away=away_version(result_matrix)
away_results=away.copy()

for i in range(0,20):
    for j in range(0,20):
        if i==j: 
            away_results[i,j]=0
        elif i!=j and away[i,j]>=0:
            away_results[i,j]=1+2*away[i,j]
        elif i!=j and away[i,j]<0:
            away_results[i,j]=1+away[i,j]


print(home_results[1])
print(result_matrix[1]-train_matrix[1])
print(away_results[1])
print(away[1])
for i in range(0,20):
    print(sum(home_results[i]+away_results[i]))


[  3.83093575e-01   0.00000000e+00   4.41302108e-04   3.00070584e+00
   1.00057435e+00   7.21489097e-01   5.55189223e-01   2.99722463e+00
   8.85184327e-06  -1.90132484e-03   1.64421344e-04  -2.62240187e-01
   3.00073506e+00   3.00049054e+00  -5.22531093e-01   4.42718655e-04
   1.36209245e-04   2.99911050e+00   9.99079888e-01   5.69251390e-01]
[[             nan              nan   4.41302108e-04   3.52918972e-04
    2.87174539e-04              nan              nan  -1.38768314e-03
    8.85184327e-06  -1.90132484e-03   1.64421344e-04              nan
    3.67531620e-04   2.45269302e-04              nan   4.42718655e-04
    1.36209245e-04  -4.44747929e-04  -9.20112326e-04              nan]]
[  2.08976058e-04   0.00000000e+00  -7.86166652e-04   1.00056216e+00
   2.99933242e+00  -1.99669594e-04  -6.93740457e-04   1.49598733e-01
   6.48294431e-04   1.03771741e-01   6.16948090e-04   1.00028980e+00
  -1.01694701e+00   2.99911367e+00   7.14401418e-04  -1.52366196e-04
   3.12302593e-05   4.6137